In [1]:
import os
import numpy as np
import pandas as pd
from nptdms import TdmsFile
import matplotlib.pyplot as plt
from scipy.signal import decimate, find_peaks, peak_widths
from scipy.stats import zscore

# Load map and locate data
# N.B - to get MacOS to mount the X Drive: osascript -e 'mount volume "smb://campus.bath.ac.uk/files/ElecEng/ResearchProjects/BMetcalfe/"'
remote_path = "/Volumes/BMetcalfe/BWM-Recordings/ENG/[EPSRC] RVC/Acute book experiments/"

if not os.path.isdir(remote_path):
    raise ValueError("Given remote root folder does not exist or is not accessible!")

# The map file has the specific filenames of each TDMS file and brief notes on the experiment
map_data = pd.read_csv(os.path.join(remote_path, "map_acute_book_dermatome.csv"))

In [ ]:
# Tryout an animal with the clicker
data_file = map_data["ENG"][15]
data_path = f"{data_file}.tdms"

# Load TDMS file
for dirpath, _, filenames in os.walk(remote_path):
        if data_path in filenames:
            data_path = os.path.join(dirpath, data_path)
# Check if the file is accessible 
if not os.path.isfile(data_path):
    raise ValueError("Given TDMS file does not exist or is not accessible!")
# Load data  
data = TdmsFile.read(data_path)

# Extract the clicker track and the ENG data
# The clicker is in channel 27, and the ENG data is in channels 3-25 (odd channels)
# N.B each book has three electrodes and thus two channels, so the even channels are just the other dipole (could average or compare)
clicker = data["Data"]["MeasuredData"][27]["Data"]
eng = np.zeros((data["Data"]["MeasuredData"][3]["Total_Samples"], 12))

j = 0
for i in range(3, 25, 2):
    eng[:, j] = data["Data"]["MeasuredData"][i]["Data"]
    j += 1

# Downsample into the realm of the living
# The clicker was recorded at 100 kHz, and the ENG at 500 kHz
dt = 1/50e3
clicker = decimate(clicker, 2)
eng = decimate(eng, 10, axis=0)

# Create time vector
nt = len(eng)
t = np.arange(nt) * dt

# Extract Features (Moving Window Variance)
# The moving window variance is calculated using a rolling window of 0.5 seconds
eng = pd.DataFrame(eng).rolling(int(0.5/dt)).var().to_numpy()
eng = zscore(eng, axis=0)

# Find stimulation windows
nt = len(clicker)
t = np.arange(nt) * dt
# Detect peaks
loc, _ = find_peaks(clicker, height=1)
# Calculate peak widths
w = peak_widths(clicker, loc, rel_height=1.0)  # Full width at base
# Calculate the width of the window
win = int(np.ceil(max(w)))
data_matrix = np.zeros((win+1, 12, len(loc)))

# Extract the data around the peaks
for i in range(len(loc)):
    data_matrix[:, :, i] = eng[loc[i]:loc[i]+win, :]


In [ ]:

# Plot average A response
A = np.zeros((win+1, 12, 6))
idx = np.arange(0, 30, 5)
for i in range(6):
    A[:, :, i] = np.mean(data_matrix[:, :, idx[i]:idx[i]+5], axis=2)

# Plot average D response
D = np.zeros((win+1, 12, 5))
idx = np.arange(85, 110, 5)
for i in range(5):
    D[:, :, i] = np.mean(data_matrix[:, :, idx[i]:idx[i]+5], axis=2)
